#Basic Autoencoder

##Import libraries and get data

In [0]:
%%capture
#Install skorch package 
!pip install skorch

In [0]:
#Data managment 
import pandas as pd 
import numpy as np 

#Machine learning
import torch as t
from torch import nn, optim
import torch.nn.functional as F

#Sklearn tools
from sklearn.model_selection import train_test_split

#Import model related libraries
import skorch
from skorch import NeuralNetRegressor
from skorch.callbacks import Checkpoint, LRScheduler

#Utilities
import os

In [0]:
%%capture
#Get data
if not os.path.isfile('all_scaled0_1.csv'): 
    !wget 'https://drive.google.com/uc?export=download&id=1-ET9vXPKudU92XuWeR0wIL67byS2llq-' -O all_scaled0_1.csv
if not os.path.isfile('data_final_feat_scaled0_1.csv'): 
    !wget 'https://drive.google.com/uc?export=download&id=14Qasv71GRD3kT3GhyI2v6c8v4L6YIBy0' -O data_final_feat_scaled0_1.csv

##Parse Data

In [0]:
#data_file = 'all_scaled0_1.csv'
data_file = 'data_final_feat_scaled0_1.csv'

In [5]:
data = pd.read_csv(data_file,index_col=0)
labels = data['label']
run = data['run']
chunkID = data['chunkID']
period = data['period']
data = data.drop(['label','chunkID','run','period'], axis = 1)
no_samples, no_features = data.shape
print(no_samples, no_features)
print(labels.value_counts())
data.head()

3429 128
0.0    3262
1.0     167
Name: label, dtype: int64


,dcar_posA_2_Err,slopedZAchi2Pos,MIPattachSlopeC,meanMultNeg,mediumPtAPos,dcarCP0,dcaz_negA_1_Err,tpcConstrainPhiA,dcar_negA_chi2,offsetdZC,dcaz_negA_0_Err,zPullHighPt,vertOK,tpcItsMatchHighPtA,deltaPt,resolutionMIP,entriesVertY,entriesMult,dcaz_posA_0,dcar_posC_1,slopedRAchi2Pos,tpcConstrainPhiC,iroc_A_side,meanVertZ,slopedZAchi2,deltaPtC,offsetdZAErr,offsetdZA,dcaz_posA_2_Err,dcaz_negC_0_Err,offsetdRA,meanMIPele,offsetdZAchi2Pos,medianHVandPTGainCorrIROC,iroc_C_side,vertAll,dcar_negA_1_Err,offsetdRAchi2Pos,meanMult,qOverPtC,...,phiPull,rmsTPCChi2,dcar_negC_1_Err,offsetdRCchi2,meanTPCChi2,rmsVertY,tpcItsMatchHighPtC,dcaz_negC_2_Err,yPullHighPt,tpcItsMatchC,deltaPtchi2C,deltaPtchi2,rmsMultNeg,meanTPCnclF,ptPullHighPt,oroc_C_side,slopedZCchi2Pos,rmsTPCncl,dcar_posC_2_Err,offsetdZCchi2Pos,deltaPtchi2A,resolutionMIPele,dcarCP1,dcar_posA_1_Err,slopedRCchi2Pos,slopeCTPCnclFErr,slopeCTPCnclF,dcar_negC_0_Err,slopedZC,lambdaPull,dcaz_posC_1_Err,offsetdRCchi2Neg,dcaz_negC_chi2,offsetdRCchi2Pos,dcaz_posC_0_Err,oroc_A_side,offsetdZAchi2,dcar_negA_0_Err,dcar_negC_chi2,slopeATPCnclF
0,0.084978,0.018955,0.842762,0.171907,0.269421,0.012700,0.073086,0.365811,0.040081,0.456515,0.075152,0.494765,0.072495,0.573832,0.778597,0.094750,0.072495,0.072495,0.402053,0.832478,0.017025,0.029029,1.0,0.314548,0.066360,0.607326,0.029047,0.676500,0.080796,0.064750,0.827629,0.539362,0.018955,0.643058,1.0,0.072211,0.082068,0.017025,0.170502,0.999787,...,0.279488,0.299807,0.089595,0.165264,0.586648,0.111019,0.582128,0.066403,0.397318,0.586826,0.232100,0.210562,0.220209,0.650378,0.478282,1.0,0.103951,0.130696,0.062340,0.103951,0.254022,0.377138,0.863747,0.091667,0.166519,0.506033,0.506033,0.092639,0.479150,0.564013,0.063305,0.169389,0.049049,0.166519,0.060923,1.0,0.066360,0.089056,0.096599,0.567304
1,0.083858,0.078152,0.828729,0.173973,0.240232,0.013681,0.072295,0.279536,0.029575,0.656468,0.074312,0.458420,0.074775,0.549569,0.745472,0.091434,0.074775,0.074775,0.388408,0.802137,0.005406,0.084570,1.0,0.298703,0.098105,0.597697,0.028192,0.672718,0.079918,0.062965,0.794622,0.539610,0.078152,0.644604,1.0,0.074384,0.080961,0.005406,0.171713,0.999787,...,0.295763,0.299619,0.087815,0.277622,0.579001,0.115167,0.581842,0.064488,0.428699,0.583827,0.149810,0.385217,0.236852,0.652363,0.508984,1.0,0.195836,0.129801,0.061326,0.195836,0.515967,0.377678,0.864576,0.090594,0.270753,0.490807,0.490807,0.090666,0.622429,0.559882,0.061817,0.295496,0.053690,0.270753,0.059619,1.0,0.098105,0.088029,0.080482,0.553643
2,0.083916,0.144460,0.829774,0.176758,0.276467,0.013235,0.071540,0.319029,0.040652,0.774050,0.073681,0.461882,0.075070,0.567858,0.739555,0.089755,0.075070,0.075069,0.394434,0.787007,0.014864,0.133775,1.0,0.281690,0.102944,0.589850,0.027634,0.709053,0.079374,0.062983,0.800667,0.539159,0.144460,0.644119,1.0,0.074775,0.080976,0.014864,0.175191,0.999787,...,0.281763,0.302841,0.087507,0.349164,0.582115,0.119110,0.586257,0.064412,0.420849,0.584458,0.421557,0.435664,0.234255,0.656779,0.493338,1.0,0.224404,0.128520,0.061049,0.224404,0.247578,0.377654,0.864001,0.090476,0.332814,0.486826,0.486826,0.090231,0.716557,0.561373,0.061571,0.391960,0.070414,0.332814,0.059297,1.0,0.102944,0.088169,0.067023,0.576802
3,0.085152,0.084640,0.835028,0.174482,0.258437,0.016565,0.072717,0.251844,0.024053,0.865839,0.074933,0.496001,0.073988,0.541670,0.761264,0.091761,0.073988,0.073987,0.389906,0.784608,0.016920,0.160170,1.0,0.291844,0.074307,0.600640,0.029305,0.684981,0.080639,0.064623,0.779825,0.539396,0.084640,0.637938,1.0,0.073704,0.081358,0.016920,0.172629,0.999787,...,0.270435,0.307098,0.088426,0.427887,0.590880,0.113392,0.589802,0.066359,0.439337,0.572863,0.565959,0.406805,0.229296,0.650338,0.537189,1.0,0.224069,0.127972,0.062087,0.224069,0.425016,0.377537,0.863877,0.091592,0.391857,0.465535,0.465535,0.091349,0.786867,0.529956,0.063318,0.491013,0.095369,0.391857,0.060874,1.0,0.074307,0.088482,0.065335,0.573718
4,0.083390,0.070673,0.833388,0.174964,0.265868,0.016429,0.071677,0.286252,0.048721,0.7102

##Split Data
If **train_on_all** is true labeled outliers are also used in training. If not then:

**Test:** All outliers and same amount of randomly choosen correct samples

**Training:** Rest of all correct samples

In [0]:
#Include outliers in training? 
train_on_all = True
size_of_test = 0.2 #When traning only on correct samples it will be this % of corr and all outliers.

In [7]:
if train_on_all:
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=size_of_test,stratify=labels)
else:
    indices_f = np.where(labels)[0] # Get inidices of flagged samples
    data_only_corr = data.drop(index = indices_f) # Delete flagged samples 
    indices_nf = np.random.choice(data_only_corr.index, round(no_samples * size_of_test), replace=False) # Get indicies of correct samples for test
    indices_test = np.concatenate([indices_f, indices_nf]) #Get all test indices

    X_test = data[data.index.isin(indices_test)]
    y_test = labels[labels.index.isin(indices_test)]
    X_train = data_only_corr.sample(frac=1).reset_index(drop=True)

print(y_test.value_counts())
print(X_train.shape)

0.0    686
1.0    167
Name: label, dtype: int64
(3262, 128)


##Define Module Architecture

Encoder and Decoder separatly just in case.

In [0]:
class Encoder(nn.Module):
    def __init__(self, no_features, lat_r, dropout):
        super().__init__()
        NF = no_features
        self.fc1 = nn.Linear(NF, round(lat_r[0]*NF))
        self.fc2 = nn.Linear(round(lat_r[0]*NF), round(lat_r[1]*NF))
        self.fc3 = nn.Linear(round(lat_r[1]*NF), round(lat_r[2]*NF))
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(self.fc3(x)) #Only droput on last layer no ReLu
        return x

class Decoder(nn.Module):
    def __init__(self, no_features, lat_r, dropout):
        super().__init__()
        NF = no_features
        self.fc1 = nn.Linear(round(lat_r[2]*NF), round(lat_r[1]*NF))
        self.fc2 = nn.Linear(round(lat_r[1]*NF), round(lat_r[0]*NF))
        self.fc3 = nn.Linear(round(lat_r[0]*NF), NF)
        self.dropout = nn.Dropout(p=dropout)       
        
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = t.sigmoid(self.fc3(x))
        return x

In [0]:
class SimpleAutoEncoder(nn.Module):
    def __init__(self, no_features, lat_r, dropout):
        super().__init__()

        self.encoder = Encoder(no_features, lat_r, dropout)
        self.decoder = Decoder(no_features, lat_r, dropout)
        
    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded

##Check GPU aviability

In [10]:
# check if CUDA is available
gpu = t.cuda.is_available()
if not gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


##Set parameters and define model

In [0]:
#Parameters setting
max_epochs = 70
dropout = 0.2
batch_size = 32
start_lr = 0.007 #Starting learning rate 

#Learning Rate scheduler
patience = 5 #After how many epochs with no improvemnt change LR
factor = 0.8 #Scale factor of LR 

#Encoding
latent_ratio = [0.5, 0.3, 0.2] #Must be 3 elements

In [0]:
#Create callbacks
model_path = 'skorch_ae_model.pt'
checkpoint = Checkpoint(f_params=model_path, monitor='valid_loss_best')
scheduler = LRScheduler(policy='ReduceLROnPlateau', monitor='valid_loss', factor = factor, patience = patience, threshold=1e-3) 

net = NeuralNetRegressor(
    SimpleAutoEncoder,
    module__no_features = no_features,
    module__lat_r = latent_ratio,
    module__dropout = dropout,
    batch_size = batch_size,
    lr=start_lr,
    optimizer = optim.Adam,
    max_epochs=max_epochs,
    train_split=skorch.dataset.CVSplit(3),
    device='cuda' if gpu else 'cpu',
    callbacks=[checkpoint, scheduler],
    verbose = 1
)
#Default criterion is <class 'torch.nn.modules.loss.MSELoss'>

##Train Autoencoder

In [13]:
X = t.FloatTensor(X_train.to_numpy())
_ = net.fit(X,X) #output compared with imput, hide output

  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.0179        0.0068     +  0.3318
      2        0.0073        0.0068     +  0.2556
      3        0.0071        0.0067     +  0.2443
      4        0.0070        0.0067     +  0.2609
      5        0.0063        0.0044     +  0.2518
      6        0.0048        0.0038     +  0.2599
      7        0.0043        0.0035     +  0.2607
      8        0.0041        0.0034     +  0.2768
      9        0.0043        0.0034     +  0.2685
     10        0.0040        0.0032     +  0.2697
     11        0.0040        0.0033        0.2864
     12        0.0038        0.0032     +  0.2571
     13        0.0038        0.0033        0.2595
     14        0.0036        0.0029     +  0.2533
     15        0.0035        0.0029     +  0.2513
     16        0.0034        0.0027     +  0.2666
     17        0.0035        0.0029        0.2671
     18        0.0033        0.0028        0.2546


##Test model

In [0]:
#Define function calculating averge losses
def aeTestLosses(X_test, X_pred, y_test):
    '''
    X_test, X_pred need to be tensors!
    Return average MSE of OUTLIERS and CORRECT samples in this order.
    '''
    outliers = []
    correct = []
    loss_func = nn.MSELoss()
    for t,p,l in zip(X_test, X_pred, y_test):
        if l: #If outlier
            outliers.append(loss_func(t,p))
        else:
            correct.append(loss_func(t,p))
    #Return calculated averages as normal numbers        
    return (sum(outliers)/len(outliers)).item() , (sum(correct)/len(correct)).item()

**Load best model from checkpoint**

In [15]:
#Load model with lowest validation accuracy
net.initialize()  # This is important!
net.load_params(model_path)

Re-initializing module because the following parameters were re-set: dropout, lat_r, no_features.
Re-initializing optimizer.


**Run prediction and test** 

In [16]:
X_test_t = t.FloatTensor(X_test.to_numpy()) #Test data to tensor
pred = net.forward(X_test_t)
o,c = aeTestLosses(X_test_t, pred, y_test)

print('Score of AE: {:0.2f}mu'.format((o-c)*1000))

Score of AE: 53.90mu
